# Dataset Generation
by [Lei You](http://user.it.uu.se/~leiyo378)

In this document, we generate the dataset, i.e., for the formulation below
$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} \\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1 \quad i\in\mathcal{I} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$
we generate the parameters $\mathcal{I}$, $\mathcal{B}$, $\mathcal{K}^{(\ell)}$, $\mathcal{K}^{(\ell)}$, $\mathbf{r}$, $\mathbf{q}$, $\mathbf{a}$.

In [476]:
import csv
import numpy
import scipy.io
from operator import add

We set the default time length and frequency bandwidth.

In [477]:
time_length = 2*1e-3 # in s
frequency_length = 2000 # in kHz

subcarrier_bw = 15 # subcarrier bandwidth (kHz)
unit_tl = 0.5*1e-3 # in s
unit_fl = 15*12 # in kHz

num_fq_unit = int(frequency_length/unit_fl)
num_tm_unit = int(time_length/unit_tl)

snr_mat = numpy.matrix(scipy.io.loadmat("snr.mat")['Expression1'])
if snr_mat.shape[0] != num_fq_unit:
    print 'row dimension of SNR matrix does not match the frequency'

In [478]:
num_fq_unit

11

In [479]:
# # vector q, only for Kl
q = [ q_elem * 1024 * 1.024 for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbps

numpy.savetxt("q.csv", q, delimiter=",")

In [480]:
num_latency = len(q) # the number of services constrained by latency
num_capacity = 10-num_latency # the number of services to maximize capacity
num_service = num_latency + num_capacity # total number of services

In [481]:
tti = [ x * 1e-3 for x in [0.5, 0.25, 0.125, 0.125, 0.15] ]

num_of_shapes = len(tti)

subcarrier_num = [ 12, 24, 48, 48, 12] # one subcarrier is 15 kHz
shape_bw = [ n*subcarrier_bw for n in subcarrier_num ]
num_shape_fq_unit = [ int(bw/unit_fl) for bw in shape_bw ]
num_shape_tm_unit = [ int(tl/unit_tl) for tl in tti ] 

In [482]:
tau = [ t*time_length for t in [1.0, 0.7, 0.8, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] ]

The vector $\mathbf{q}$ is set as follows.

The sets $\mathcal{K}$, $\mathcal{K}^{(\ell)}$, and $\mathcal{K}^{(c)}$ are generated as follows.

In [483]:
# sets of services
Kl = range(num_latency)
Kc = range(num_latency,num_service)
K = Kl + Kc

numpy.savetxt("Kl.csv", Kl, delimiter=",")
numpy.savetxt("Kc.csv", Kc, delimiter=",")

In [484]:
with open('SNR_list.csv', 'rb') as f:
    snr_csv = csv.reader(f)
    snr_table = list(snr_csv)
    snr_table = [item for sublist in snr_table for item in sublist] # flatten list
    snr_table = map(float, snr_table) # convert to float

In [485]:
with open('DataVol_list.csv', 'rb') as f:
    datavol_csv = csv.reader(f)
    datavol_table = list(datavol_csv)
    datavol_table = [ map(float,x) for x in datavol_table] # convert to float
    datavol_table = map(list, zip(*datavol_table)) # transpose table

We do interplotation for the two tables.

In [486]:
from scipy.interpolate import interp1d

# interplotation for snr_table
x = snr_table
y = range(1,31)
snr_interpld = interp1d(x, y)

In [487]:
# interplotation for datavol_table
x = range(1,31)
y1 = datavol_table[0] # for shape 1
y2 = datavol_table[1] # for shape 2
y3 = datavol_table[2] # for shape 3
y4 = datavol_table[3] # for shape 4
y5 = datavol_table[4] # for shape 5

datavol_interpld = [interp1d(x,y1), interp1d(x,y2), interp1d(x,y3), interp1d(x,y4), interp1d(x,y5)]

The function that computes the data volume for a given shape and average SNR is implemented as follows.

In [488]:
def shapeCapacity(PRB_shape, avg_snr): 
    if PRB_shape < 0 or PRB_shape > 4:
        print 'shape_index is out of range: valid index is between 0--4'
        return
    return datavol_interpld[PRB_shape](avg_snr)  # capacity is in bit

The set $\mathcal{B}$ is generated as follows.

In [489]:
# The function returns a tuple: The first element is the set B; 
#                               The second is the number of each shape in B
def getB():
    n_frequency = [ num_fq_unit - num_shape_fq_unit[i] + 1 for i in range(num_of_shapes) ]
    n_time = [ num_tm_unit - num_shape_tm_unit[i] + 1 for i in range(num_of_shapes) ]
    n = [ int(a*b) for a,b in zip(n_frequency,n_time) ]
    
    B1 = range(n[0]) # indices of shape 1
    B2 = range(n[0],n[0]+n[1]) # indices of shape 2
    B3 = range(n[0]+n[1],n[0]+n[1]+n[2]) # indices of shape 3 
    B4 = range(n[0]+n[1]+n[2],n[0]+n[1]+n[2]+n[3]) # indices of shape 4
    B5 = range(n[0]+n[1]+n[2]+n[3], n[0]+n[1]+n[2]+n[3]+n[4]) # indices of shape 5
    
    B = B1 + B2 + B3 + B4 + B5
    return [B, n]

In [490]:
tmp = getB()
B = tmp[0]
nB = tmp[1]

numpy.savetxt("B.csv", B, delimiter=",")
numpy.savetxt("nB.csv", nB, delimiter=",")

The following function gets the set $\mathcal{I}$.

In [491]:
# The function returns I
def getI():
    n = num_fq_unit * num_tm_unit
    return range(n)

In [492]:
I = getI()

numpy.savetxt("I.csv", I, delimiter=",")

The following function calculates the matrix $\mathbf{a}$. We use a function named $\texttt{unitInPRB}$ to check whether a resource unit belongs to a PRB. The function $\texttt{unitInPRB}$ returns $\texttt{True}$ if the resource unit locates inside the PRB. 

In [493]:
def getPRBShape(PRB_index):
    if PRB_index < nB[0]:
        PRB_shape = 0
    elif PRB_index < nB[0]+nB[1]:
        PRB_shape = 1
    elif PRB_index < nB[0]+nB[1]+nB[2]:
        PRB_shape = 2
    elif PRB_index < nB[0]+nB[1]+nB[2]+nB[3]:
        PRB_shape = 3
    else:
        PRB_shape = 4
    return PRB_shape

In [494]:
def unitInPRB(PRB_index, unit_index):
    
    # compute the indices of row and column for the given index of resource unit
    unit_row = unit_index / num_tm_unit # starting from 0 
    unit_column = unit_index % num_tm_unit # starting from 0
    
    # compute the indices range of row and column for the given index of PRB
        # we first compute the "row position" and "column position" in terms of 
        # PRB shape as we do in function "getB"
    PRB_shape = getPRBShape(PRB_index)
    
    shifted_PRB_index = PRB_index
    for i in range(PRB_shape):
        shifted_PRB_index -= nB[i]
    
    num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
    num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
        
        # we then compute the range of row and column for the PRB
    PRB_row_start = shifted_PRB_index / num_time  # starting from 0
    PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
    PRB_column_start = shifted_PRB_index % num_time
    PRB_column_end = PRB_column_start + num_shape_tm_unit[PRB_shape] - 1
    
    if unit_row >= PRB_row_start and unit_row <= PRB_row_end and unit_column >= PRB_column_start and unit_column <= PRB_column_end:
        return True
    else:
        return False

The following function computes the matrix $\mathbf{a}$.

In [495]:
def geta():
    a = [ [ 0 for i in I ] for b in B ]
    for b in B:
        for i in I:
            if unitInPRB(b,i) == True:
                a[b][i] = 1
    return a

In [496]:
a = geta()

numpy.savetxt("a.csv", a, delimiter=",")

The following code calculates a matrix for PRB confliction relationship.

In [497]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
conflict_PRB = [ [ 0 for b in B ] for b in B ]
for b1 in range(len(B)-1):
    conflict_PRB[b1][b1] = 1
    for b2 in range(b1+1,len(B)):
        if b1!=b2 and max(map(add,a[b1],a[b2]))>1: # there is at least one RU overlap between b1 and b2
            conflict_PRB[b1][b2]=conflict_PRB[b2][b1]=1
conflict_PRB[len(B)-1][len(B)-1] = 1

numpy.savetxt("conflict_PRB.csv", conflict_PRB, delimiter=",")

The following function gets the matrix $\mathbf{r}$.

In [498]:
def getr(snr_mat):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in B:
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [499]:
r = getr(snr_mat)

numpy.savetxt("r.csv", r, delimiter=",")